In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    accuracy_score,
)
from sklearn.pipeline import Pipeline
import pickle

# Neural Network MLP Classifier

In this section, we delve into the realm of artificial neural networks by exploring the Multi-Layer Perceptron (MLP) Classifier. MLP is a class of feedforward artificial neural network that consists of multiple layers of nodes, each layer fully connected to the next one. It is designed to model complex patterns in data by learning multiple layers of representation.

## Key Characteristics of Neural Network MLP:

- **Layered Architecture**: MLPs consist of an input layer, several hidden layers, and an output layer, allowing the model to learn at various levels of abstraction.
- **Nonlinear Activation**: Nonlinear activation functions within neurons enable the network to capture complex relationships between features.
- **Backpropagation**: MLP uses backpropagation for training, efficiently adjusting weights in the network through gradient descent.

MLPs are powerful tools capable of modeling complex and high-dimensional data, making them suitable for a wide range of classification tasks, from image recognition to natural language processing.

The objective of this notebook is to develop, fine-tune, and assess an MLP Classifier's performance, preparing it for subsequent comparison with other classification models in a broader study.

For an in-depth walkthrough of our setup process, please refer to the notebook "Final_Project_Data_Gen."

---


Let's load our dataset from CSV files, with `X_train` and `y_train` as our training features and labels, and `X_test` and `y_test` for testing, using Pandas DataFrames to facilitate data manipulation and analysis.


In [2]:
X_train = pd.read_csv("train_X_In-Car-Rec.csv")
y_train = pd.read_csv("train_y_In-Car-Rec.csv")
X_test = pd.read_csv("test_X_In-Car-Rec.csv")
y_test = pd.read_csv("test_y_In-Car-Rec.csv")

We define a `param_grid` for fine-tuning our Multi-Layer Perceptron (MLP) classifier, specifying different architectures for `hidden_layer_sizes` to test single and multiple layers of various sizes. We also explore several `activation` functions to determine which helps our network learn best, and we vary `max_iter` to give our model sufficient iterations to converge during training.


In [3]:
param_grid = {
    "mlp__hidden_layer_sizes": [(100,), (50, 50), (50, 25, 10), (40, 20, 10)],
    "mlp__activation": ["logistic", "tanh", "relu"],
    "mlp__max_iter": [1000, 5000, 10000],
}

Here is where we will establish our custom `scorer`. We have decided to use a custom F2 score to prioritize recall (minimizing the chance of not giving a coupon to someone who would use it) while considering the value of precision (minimizing the chance of giving a coupon to someone who will not use it).


In [4]:
# Defining a function
scorer = make_scorer(fbeta_score, beta=2)

We create a comprehensive pipeline named `pipeline` that encapsulates the MLPClassifier. This setup streamlines the process of standardizing our workflow, making it straightforward to include additional preprocessing steps in the future if necessary.


In [5]:
# Create the full pipeline
pipeline = Pipeline([("mlp", MLPClassifier())])

We instantiate a `GridSearchCV` object that applies an exhaustive search over the specified `param_grid` for our neural network pipeline. By setting `cv=3`, we use 3-fold cross-validation to assess each set of parameters, and we specify our performance metric with the `scoring` object. Finally, we fit this grid search to our training data, allowing us to identify the best hyperparameter combination for our MLPClassifier.


In [6]:
# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring=scorer)

# Fit the pipeline (including hyperparameter tuning) to your data
grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=3, estimator=Pipeline(steps=[('mlp', MLPClassifier())]),
             param_grid={'mlp__activation': ['logistic', 'tanh', 'relu'],
                         'mlp__hidden_layer_sizes': [(100,), (50, 50),
                                                     (50, 25, 10),
                                                     (40, 20, 10)],
                         'mlp__max_iter': [1000, 5000, 10000]},
             scoring=make_scorer(fbeta_score, beta=2))

Now we will retrieve and store the best-performing model from our grid search in the variable `best_estimator`. Additionally, we extract the optimal set of hyperparameters and the highest cross-validation score obtained during the search, assigning them to `best_random_params` and `best_random_score` respectively. These variables give us insight into the most effective configuration of our MLPClassifier and its performance.


In [7]:
# Store best estimator
best_estimator = grid_search.best_estimator_

# Get the best parameters and score
best_random_params = grid_search.best_params_
best_random_score = grid_search.best_score_

best_random_params, best_random_score

({'mlp__activation': 'logistic',
  'mlp__hidden_layer_sizes': (100,),
  'mlp__max_iter': 1000},
 0.7743742278414341)

We construct our final pipeline, `pipeline_MLP`, which is configured with an `MLPClassifier`. The classifier is set with a logistic activation function, a single hidden layer of 100 neurons, and a maximum of 1000 iterations for the training process. This pipeline encapsulates our chosen model configuration for the MLP, ready for training and subsequent evaluation.


In [8]:
# final pipeline
pipeline_MLP = Pipeline(
    [
        (
            "mlp",
            MLPClassifier(
                activation="logistic",
                hidden_layer_sizes=(100,),
                max_iter=1000,
            ),
        ),
    ]
)

Now it is time to train our final MLP pipeline, `pipeline_MLP`, using the training data `X_train` and the target values `y_train`. By using the `ravel()` method, we ensure that the `y_train` data is in the correct shape for the fitting process, which is a one-dimensional array.


In [9]:
# Train the final pipeline
pipeline_MLP.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('mlp', MLPClassifier(activation='logistic', max_iter=1000))])

Let's make predictions on the test dataset `X_test`. The predicted labels for the test data are stored in `y_pred`, which we will use for evaluating the model's performance.


In [10]:
# Predict on the test set
y_pred = pipeline_MLP.predict(X_test)

We evaluate the performance of our MLP pipeline on the test data `X_test` by using the `score` method, which provides the accuracy of the model. Additionally, we calculate the weighted F2 score, which places more emphasis on recall, using the `fbeta_score` function with `beta=2`. This score is particularly useful for imbalanced classes. The calculated F2 score is then printed to give us a more nuanced understanding of our model's predictive capability.


In [11]:
# Evaluate the pipeline on the test data
score = pipeline_MLP.score(X_test, y_test)

# Calculate f1_score on the test data
f2_score = fbeta_score(y_test, y_pred, average="weighted", beta=2)
print(f"F2Score for the MLP Classifier Model is: " + str(f2_score))

F2Score for the MLP Classifier Model is: 0.7412090067455147


The F2Score for our MLP Classifier Model is approximately 0.7412, indicating a model performance that balances precision and recall, with an emphasis on recall. This suggests that our model is adept at correctly identifying positive instances in the test data, particularly valuable in situations where minimizing false negatives is crucial.


We perform a comprehensive evaluation of our MLP Classifier model's performance:

- **Confusion Matrix**: We calculate a confusion matrix to visualize the model's predictions and actual outcomes, providing insights into true positives, true negatives, false positives, and false negatives.

- **Precision (weighted)**: We compute the weighted precision, which gives us an overall measure of how often the model's positive predictions are correct, accounting for class imbalance.

- **Recall (weighted)**: The weighted recall is determined, reflecting the model's ability to correctly identify positive instances while considering class distribution.

- **Accuracy**: We calculate the overall accuracy of the model, representing the proportion of correct predictions among all predictions made.

These metrics collectively provide a comprehensive view of the MLP Classifier model's effectiveness in making predictions on the test data.


In [12]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision
precision = precision_score(y_test, y_pred, average="weighted")
print(f"\nPrecision (weighted): {precision:.4f}")

# Recall
recall = recall_score(y_test, y_pred, average="weighted")
print(f"Recall (weighted): {recall:.4f}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Confusion Matrix:
[[ 769  309]
 [ 348 1111]]

Precision (weighted): 0.7425
Recall (weighted): 0.7410
Accuracy: 0.7410


The Confusion Matrix shows that our MLP Classifier model made 769 true negative predictions, 1111 true positive predictions, 309 false positive predictions, and 348 false negative predictions.

The weighted Precision, measuring the accuracy of positive predictions, is approximately 0.7425. The weighted Recall, which assesses the model's ability to identify positive instances, stands at approximately 0.7410.

The overall Accuracy of our model is approximately 0.7410, indicating that it correctly predicts around 74.10% of instances in the test data. These metrics collectively provide an understanding of our model's performance on the test dataset.


## Pickling Our Model


In [13]:
# Specify the filename where you want to save the model
filename = "MLP_Model.pkl"

# Export the model to the file using pickle.dump
with open(filename, "wb") as file:
    pickle.dump(pipeline_MLP, file)